#Delta Sharing setup notebook
This notebook will create a catalog for the demo and add the Delta tables to this demo so that we can share them

In [0]:
dbutils.widgets.dropdown("reset_all_data", "false", ["true", "false"], "Reset all data")
reset_all_data = dbutils.widgets.get("reset_all_data") == "true"

In [0]:
catalog = "pds"
schema = dbName = db = "dbdemos_sharing_airlinedata"

volume_name = "raw_data"

In [0]:
%run ./00-global-setup-v2

In [0]:
DBDemos.setup_schema(catalog, db, reset_all_data, volume_name)
volume_folder =  f"/Volumes/{catalog}/{db}/{volume_name}"

In [0]:
def download_recipient_credential(recipient, location):
  import urllib.request
  sql(f"""DROP RECIPIENT {recipient}""")
  sql(f"""CREATE RECIPIENT {recipient}""")
  if recipient == 'dbdemos_americanairlines_recipient':
      sql('''ALTER RECIPIENT dbdemos_americanairlines_recipient SET PROPERTIES ('carrier_id' = 'AA');''')
  else:
      sql('''ALTER RECIPIENT dbdemos_southwestairlines_recipient SET PROPERTIES ('carrier_id' = 'WN');''')
      
  df = sql(f"""DESCRIBE RECIPIENT {recipient}""")
  if 'info_name' in df.columns:
    link = df.where("info_name = 'activation_link'").collect()[0]['info_value']
  else:
    link = df.collect()[0]['activation_link']
  if link is not None:
    link = link.replace('delta_sharing/retrieve_config.html?','api/2.0/unity-catalog/public/data_sharing_activation/')
    urllib.request.urlretrieve(link, f"/tmp/{recipient}.share")
    import shutil
    try:
        dbutils.fs.mv(f"file:/tmp/{recipient}.share", location)
    except:
        shutil.copy(f"/tmp/{recipient}.share", location)
    print(f"Your file was downloaded for the demo to: {location} - BE CAREFUL PROTECT THIS FILE")

In [0]:
#Try to delete all shares and recipients + any credential remaining
def cleanup_demo():
    def try_safe(func):
        try:
            func()
        except Exception as e:
            print(f'Cleanup error: {e}')

    try_safe(lambda: dbutils.fs.rm('/Volumes/pds/dbdemos_sharing_airlinedata/raw_data/southwestairlines.share'))
    try_safe(lambda: dbutils.fs.rm('/Volumes/pds/dbdemos_sharing_airlinedata/raw_data/americanairlines.share'))
    try_safe(lambda: spark.sql('DROP RECIPIENT  dbdemos_southwestairlines_recipient'))
    try_safe(lambda: spark.sql('DROP RECIPIENT  dbdemos_americanairlines_recipient'))
    try_safe(lambda: spark.sql('DROP SHARE  dbdemos_americanairlines'))
    try_safe(lambda: spark.sql('DROP SHARE  dbdemos_southwestairlines'))

In [0]:
def get_codes():
    return  ["04Q,Tradewind Aviation",
    "05Q,Comlux Aviation, AG",
    "06Q,Master Top Linhas Aereas Ltd.",
    "07Q,Flair Airlines Ltd.",
    "09Q,Swift Air, LLC d/b/a Eastern Air Lines d/b/a Eastern",
    "0BQ,DCA",
    "0CQ,ACM AIR CHARTER GmbH",
    "0FQ,Maine Aviation Aircraft Charter, LLC",
    "0GQ,Inter Island Airways, d/b/a Inter Island Air",
    "0HQ,Polar Airlines de Mexico d/b/a Nova Air",
    "0J,JetClub AG",
    "0JQ,Vision Airlines",
    "0LQ,Metropix UK, LLP.",
    "0MQ,Multi-Aero, Inc. d/b/a Air Choice One",
    "0OQ,Open Skies",
    "0Q,Flying Service N.V.",
    "0QQ,TAG Aviation (UK) Ltd.",
    "0RQ,TAG Aviation Espana S.L.",
    "0TQ,Corporatejets, XXI",
    "0UQ,Comlux Malta, Ltd.",
    "0VQ,Ocean Sky (UK) Limited",
    "0WQ,Avjet Corporation",
    "0YQ,Comlux Malta Ltd.",
    "10Q,Swiss Air Ambulance",
    "12Q,Unijet",
    "13Q,Chartright Air Inc.",
    "14Q,London Air Services Limited",
    "15Q,Air Alsie A/S",
    "17Q,Albinati Aeronautics SA",
    "1AQ,Via Airlines d/b/a Charter Air Transport",
    "1BQ,Eastern Airlines f/k/a Dynamic Airways, LLC",
    "1DQ,Island Airlines LLC",
    "1EQ,KaiserAir, Inc.",
    "1FQ,Alsek Air",
    "1HQ,International Jet Management GmbH",
    "1I,Sky Trek International Airlines",
    "1IQ,Jet link AG",
    "1JQ,Exec Direct Aviation Services Limited",
    "1LQ,Bertelsmann Aviation GmbH",
    "1MQ,Twin Cities Air Service LLC",
    "1NQ,JSC The 224th Flight Unit State Airlines",
    "1PQ,Makani Kai Air Charters",
    "1QQ,City Wings Inc dba Seaflight",
    "1RQ,Sun Air Express LLC dba Sun Air International",
    "1SQ,Star Marianas Air Inc.",
    "1TQ,Rhoades Aviation dba Transair",
    "1UQ,Gainjet Aviation SA",
    "1WQ,Ultimate JetCharters LLC dba Ultimate Air Shuttle",
    "1XQ,Dassault Falcon Service",
    "1YQ,Air Charter, Inc d/b/a Air Flamenco",
    "1ZQ,Comlux Aruba NV",
    "20Q,Orange Air",
    "22Q,Gem Air LLC",
    "23Q,Songbird Airways Inc.",
    "24Q,Blue Jet SP Z o o",
    "25Q,Mjet GmbH",
    "26Q,Albinati aviation Limited",
    "27Q,Jet Aviation Flight Services, Inc.",
    "28Q,Menagerie Enterprises Inc d/b/a Monarch Air",
    "29Q,Erickson Helicopters",
    "2AQ,ABS JETS A.S",
    "2BQ,Air Medical Ltd.",
    "2E,Smokey Bay Air Inc.",
    "2EQ,J&M Alaska Air Tours, Inc. d/b/a Alaska Air Transit",
    "2F,Frontier Flying Service",
    "2GQ,Cavok Air LLC",
    "2HQ,Elite Airways LLC",
    "2JQ,Delux Public Charter LLC",
    "2K,Aerolineas Galapagos S A Aerogal",
    "2KQ,Longtail Aviation Ltd.",
    "2LQ,Tropic Ocean Airways LLC",
    "2M,Midway Express Airlines",
    "2NQ,LIMA NY Corp d/b/a Fly The Whale d/b/a Tailwind",
    "2O,Island Air Service",
    "2OQ,Talofa Airways Limited",
    "2P,Air Philippines Corp d/b/a Airphil Express, Philippines Airlines",
    "2PQ,21 Air LLC",
    "2R,Regal Air",
    "2RQ,Luxaviation S A",
    "2T,Canada 3000 Airlines Ltd.",
    "2TQ,Trans Northern Airways d/b/a Air Unlimited Charter Services",
    "2U,Valley Air Express Inc.",
    "2UQ,Svenskt Industriflyg AB",
    "30Q,Public Charters, Inc. d/b/a North Country Sky d/b/a Regional Sky d/b/a Texas Sky",
    "34Q,Polaris Aviation Solutions, LLC",
    "35Q,Aztec Worldwide Airlines, Inc. d/b/a Aztec Airways",
    "36Q,Air Hamburg Luftverkehrsgesellschaft mbH",
    "37,Zeal 320",
    "37Q,Hillwood Airways, LLC f/k/a ATX Air Services, LLC",
    "38Q,Key Lime Air Corporation",
    "3AQ,Maritime Helicopters, Inc.",
    "3BQ,Nealco Air Charter Services, Inc. d/b/a Watermakers Air",
    "3C,Regions Air, Inc.",
    "3CQ,Andrew Airways, Inc.",
    "3DQ,Rectrix Aviation, Inc.",
    "3EQ,Scott Aviation, LLC d/b/a Silver Air",
    "3F,Pacific Airways, Inc.",
    "3FQ,Western Air Charter, Inc. d/b/a Jet Edge",
    "3M,Silver Airways",
    "3S,AeroLogic GmbH",
    "3SD,Compagnie Aerienne Inter Regionale Express dba Air Antilles Express & Air Guyane",
    "3U,Sichuan Airlines Co Ltd.",
    "4B,Boutique Air",
    "4B (1),Olson Air Service",
    "4C,Aerovias de Intergracian Regional",
    "4E,British Airtours Limited",
    "4H,Belize Trans Air",
    "4M,LAN Argentina",
    "4M (1),Lan Dominica",
    "4N,Air North",
    "4O,ABC Aerolineas SA de CV dba Interjet",
    "4R,Regent Air Corporation",
    "4S,Sol Air (Aero Hunduras)",
    "4S (1),Conner Air Lines Inc.",
    "4T,Belair Airlines Ltd.",
    "4W,Warbelow",
    "4Y,Yute Air Aka Flight Alaska",
    "5B,Bellair Inc.",
    "5C,C.A.L Cargo Airlines",
    "5D,Aerolitoral",
    "5F,Arctic Circle Air Service",
    "5G,Skyservice Airlines, Inc.",
    "5G (1),Queen Air",
    "5J,Cebu Air Inc d/b/a Cebu Pacific Air",
    "5J (1),Private Jet Expeditions",
    "5L,Aerosur",
    "5V,Tatonduk Outfitters Limited d/b/a Everts Air Alaska and Everts Air Cargo",
    "5X,United Parcel Service",
    "5Y,Atlas Air Inc.",
    "6A,Aviacsa Airlines",
    "6B,TUIfly Nordic A.B.",
    "6C,Cape Smythe Air Service",
    "6F,Laker Airways Inc.",
    "6H,Israir Airlines",
    "6P,Pacific East Air Inc.",
    "6R,Aerounion Aerotransporte de Carga Union SA de CV",
    "6U,Air Ukraine",
    "6Y,Nicaraguense De Aviacion Sa",
    "7B,Krasair",
    "7C,Jeju Air Co Ltd.",
    "7F,First Air",
    "7G,MK Airlines Ltd.",
    "7G (1),Bellair Inc. (1)",
    "7H,Era Aviation",
    "7I,Insel Air International B V",
    "7L,Silk Way West Airlines",
    "7M,Aeromonterrey S.A.",
    "7N,Inland Aviation Services",
    "7P,Apa International Air S.A.",
    "7Q,Pan American World Airways Dominicana",
    "7S,Ryan Air f/k/a Arctic Transportation",
    "7S (1),Skystar International Inc.",
    "7Z,Lb Limited",
    "8C,Air Transport International",
    "8D,Dynamic Aviation, Inc. d/b/a Beacon",
    "8E,Bering Air Inc.",
    "8F,Wilburs Inc.",
    "8I,Inselair Aruba NV",
    "8J,Jetall Airways",
    "8L,Servicio Aereo Leo Lopez",
    "8N,Flagship Airlines Inc.",
    "8Q,Baker Aviation Inc.",
    "8R,Edelweiss Air Ag",
    "8V,Wright Air Service",
    "9E,Endeavor Air Inc.",
    "9J,Pacific Island Aviation",
    "9K,Cape Air",
    "9L,Colgan Air",
    "9M,Central Mountain Air",
    "9N,ANA & JP Express Co. Ltd.",
    "9R,Flagship Express Services",
    "9S,Southern Air Inc.",
    "9T,TravelspanGT, Inc. S.A.",
    "9V,Avior Airlines, C.A.",
    "9W,Jet Airways (India) Limited",
    "A0,Elysair d/b/a L'Avion",
    "A2,Cielos De Peru",
    "A4,Southern Wind Airlines",
    "A7,Air Comet S.A.",
    "A7 (1),Air 21",
    "AA,American Airlines Inc.",
    "AAA,Associated Aviation Act.",
    "AAB,Abelag Aviation AG",
    "AAB (1),Antilles Air Boats Inc.",
    "AAE,AAA Airlines",
    "AAI,Audi Air Inc.",
    "AAL,Argosy Air Lines Inc.",
    "AAP,Armstrong Air Service Inc.",
    "AAQ,Advance Air Charters",
    "AAR,Altair Airlines Inc.",
    "AAT,Air Sunshine Inc.",
    "AAV,Alpine Air",
    "AAZ,Alaska Aeronautical Indust.",
    "AB,Air Berlin PLC and CO",
    "ABH,Air Bahia",
    "ABI,Airbama Incorporated",
    "ABO,Aeronaves Boringuena Inc.",
    "ABX,ABX Air Inc",
    "AC,Air Canada",
    "ACA,Alaska Central Airways Inc.",
    "ACD,Academy Airlines",
    "ACG,Air Cargo Express Inc.",
    "ACH,Island Airlines Hawaii Inc.",
    "ACI,AAA-Action Air Carrier Inc.",
    "ACJ,Atlantic Coast Jet Inc.",
    "ACK,Air Central Inc.",
    "ACL,Check-Air Inc.",
    "ACN,Alliance Airlines",
    "ACO,Air Cargo Enterprises Inc.",
    "ACP,Air Chaparral Inc.",
    "ACQ,Aeronautica De Cancun",
    "ACS,Alamo Commuter Airlines",
    "ACT,Air Central Inc. (1)",
    "ACW,Transwest Air Express",
    "AD,Azul Linhas Aereas Brazileiras S A",
    "AD (1),Avia Leasing Company",
    "ADB,Antonov Company",
    "ADQ,Aeronaves Del Peru",
    "ADR,Adirondack Airlines",
    "ADV,Advantage Airlines",
    "AE,Air Europe Limited",
    "AEC,Astec Air East Inc.",
    "AED,Air East Of Delaware Inc.",
    "AEF,Aero Freight",
    "AEO,Aero Coach",
    "AEQ,Air Caravane Inc.",
    "AEX,Air Excursions LLC (1)",
    "AF,Compagnie Natl Air France",
    "AFA,American Flag Airlines Inc.",
    "AFE,Air Florida Express",
    "AFG,American Flight Group",
    "AFQ,Aero B Venezuela C.A.",
    "AFS,Arcata Flying Service",
    "AG,Arubaanse Luchtvaart Maatschappij NV dba Aruba Airlines",
    "AGA,Air Cargo America Inc.",
    "AGQ,Argo S. A.",
    "AHA,Air Alpha",
    "AHC,Air Hawaii",
    "AHQ,Air National",
    "AI,National Aviation Company of India Limited d/b/a Air India",
    "AIA,All Island Air",
    "AIC,Air Caribbean",
    "AIE,Air Express Inc.",
    "AIG,Airgo Inc.",
    "AIK,Air Ketchum Idaho Inc.",
    "AIM,Air Mont. Inc.",
    "AIN,Air Indiana",
    "AIQ,Norontair",
    "AIT,Airways International",
    "AIY,Alliance Air",
    "AJ,Air Micronesia Inc.",
    "AJQ,Aerodynamics Inc. d/b/a SkyValue d/b/a SkyValue Airways",
    "AKQ,Airmark Airlines Inc.",
    "AL,Skyway Airlines",
    "ALD,Allied Airlines",
    "ALE,Air-Lift Associates Inc.",
    "ALI,Brandt Air",
    "ALK,Air Link Airways",
    "ALL,All Seasons Air Pacific",
    "ALQ,Alberta Northern Airlines",
    "ALS,Air L.A.",
    "ALT,Altus Airlines",
    "ALU,Aleutian Air",
    "ALX,Atlanta Express",
    "AM,Aeromexico",
    "AMA,North American Airlines Inc.",
    "AME,Americair Inc.",
    "AMI,American Inter-Island Inc.",
    "AMQ,Ameristar Air Cargo",
    "AMR,AvAir",
    "AMT,American Air Transport",
    "AN,ADVANCED AIR, LLC",
    "ANB,Air Nebraska Inc.",
    "ANC,Atlantic Air",
    "AND,Southcentral Air Inc.",
    "ANI,Dash Air Corporation",
    "ANM,Airways Of New Mexico Inc.",
    "ANO,Metro Air Northeast Vermont",
    "ANT,Air North/Nenana",
    "AO,Aviacion Y Comercio S.A.",
    "AOI,Trans New York",
    "AON,Action Airlines",
    "AOQ,Aerovias S.A.",
    "AOR,Air New Orleans",
    "APE,Air Cape",
    "APH,Alpha Air",
    "API,Arizona Pacific Inc.",
    "APL,Appalachian Flying Service",
    "APN,Aspen Airways Inc.",
    "APQ,Alas De Transporte International",
    "APS,Air Spirit Inc.",
    "APY,Air Polynesia Inc.",
    "AQ,Aloha Airlines Inc.",
    "AQQ,Air Charter (Safa)",
    "AR,Aerolineas Argentinas",
    "ARA,Air Resorts Airlines",
    "ARB,Air Caribe International",
    "ARD,Airlift International",
    "ARI,Arizona Aero Corp.",
    "ARM,Airmarc Airlines Inc.",
    "ARN,Arnold Aviation",
    "ARO,Air Oregon",
    "ARP,Airspur Helicopters Inc.",
    "ARQ,Aerovias Colombianas Arca",
    "ARR,Arrowhead Airways Inc.",
    "ARS,Air Sedona",
    "ARZ,Arizona Airways",
    "AS,Alaska Airlines Inc.",
    "ASC,Air South Inc.",
    "ASD,Amistad Airlines",
    "ASH,Air South",
    "ASI,Alpine Air Services Inc.",
    "ASN,Air Casino",
    "ASP,Asap Air Inc.",
    "ASQ,Aerosur (1)",
    "ASR,Astor Air",
    "AST,Astro Airways",
    "ASU,Air South (1)",
    "ASW,Sunrise Airlines Inc.",
    "AT,Royal Air Maroc",
    "ATH,Air Southeast",
    "ATN,Air Atlantic Airlines Inc.",
    "ATO,Pacific National Airways",
    "ATQ,Aero Tranportes Panamenos",
    "ATS,Atlantis Airlines Inc.",
    "ATT,Aerotransit Inc.",
    "ATX,Air Carolina",
    "AUX,Austin Express Inc.",
    "AV,Aerovias Nacl De Colombia",
    "AVA,Air Vectors Airways",
    "AVC,Av Center Inc.",
    "AVG,Airvantage",
    "AVI,Aero Virgin Islands Corp.",
    "AVL,Aviation Sales",
    "AVQ,Aero Laguna S.A. De C.V.",
    "AVR,Air Vermont Inc.",
    "AVS,Aviation Services",
    "AVZ,Southern Jersey Airways",
    "AWA,Air West Airlines",
    "AWQ,Aermediterranea S.P.A.",
    "AX,Trans States Airlines",
    "AXQ,Aeron International Airlines Inc.",
    "AY,Finnair Oy",
    "AYQ,Aerodan S.A. De C.V.",
    "AZ,Compagnia Aerea Italiana",
    "AZP,Arizona Pacific Airways",
    "AZQ,Anglo Airlines Ltd.",
    "B0,Dreamjet SAS Dba La Compagnie",
    "B6,JetBlue Airways",
    "BA,British Airways Plc",
    "BAA,Aero International Airlines",
    "BAC,Business Aircraft Corp.",
    "BAH,Brennan & Hargreaves Inc.",
    "BAK,Bankair Inc.",
    "BAN,Bangor International Air",
    "BAQ,Aero Rentas De Coahuila S.A. De C.V.",
    "BAR,Bard-Air Corp.",
    "BAT,West Penn Commuter",
    "BAY,Burlington Aeroplane Co.",
    "BBQ,Balair Ag",
    "BC,Skymark Airlines Inc.",
    "BCA,Baja Cortez Airlines",
    "BCQ,Bradley Air Services Ltd.",
    "BD,British Midland Airways Ltd.",
    "BDE,Jet Express",
    "BDQ,Aerotaxis De Aguascalientes S.A. De C.V.",
    "BE,Braniff International Airlines Inc.",
    "BEA,Business Express Airlines",
    "BEN,Bennington Aviation Inc.",
    "BEQ,Air Belgium",
    "BES,Aeroservices Executive",
    "BEY,Beyer Aviation",
    "BF,Markair Inc.",
    "BFG,Million Air",
    "BFQ,Buffalo Airways Inc.",
    "BG,Biman Bangladesh Airlines",
    "BGQ,Airbus Transport Internat'l",
    "BHK,Blackhawk Airways Inc.",
    "BHO,Bighorn Airways Inc.",
    "BHQ,Turks Air Ltd. (1)",
    "BIA,Big Island Air",
    "BIQ,Servicios Aeronauticos Z S.A. De C.V.",
    "BIR,Birchwood Air Service",
    "BJQ,Taxirey S.A. De C.V.",
    "BLQ,Air Bvi Limited",
    "BM,Belize Air International Ltd.",
    "BMT,Bremerton Aviation Inc.",
    "BN,Britannia Airways GmbH",
    "BN (1),Braniff Inc.",
    "BNQ,NetJets Large Aircraft Company L.L.C.",
    "BOA,Boise Air Service",
    "BOI,Bo-S-Aire Airlines Inc.",
    "BOQ,Transp. Aereos Bolivianos",
    "BQ,Aeromar C. Por A.",
    "BR,Eva Airways Corporation",
    "BR (1),Cal Air International Ltd.",
    "BRK,Barken International",
    "BRO,Brower Airways Inc.",
    "BRW,Barrow Air",
    "BTQ,Brasair Transportes Aereos",
    "BUH,Business Helicopters",
    "BUQ,Wrangler Aviation Inc.",
    "BUQ (1),Blue Bell Inc.",
    "BUR,Burlington Airways Inc.",
    "BW,Caribbean Airlines Limited",
    "BX,Air Busan",
    "BXQ,Spantax S. A.",
    "BY,Britannia Airways Ltd.",
    "BYA,Bay Air",
    "BZQ,Abaco Air, Ltd.",
    "C5,Commutair Aka Champlain Enterprises, Inc.",
    "C6,Canjet Airlines",
    "C8,Cargolux Italia SpA",
    "C8 (1),Chicago Express Airlines",
    "CA,Air China",
    "CAC,Coleman Air Transport",
    "CAF,California Seaboard",
    "CAI,Crawford Aviation Inc.",
    "CAM,C & M Airlines",
    "CAP,Metro Air Northeast N Y",
    "CAQ,Challenge Air Luftverkehrs GmbH",
    "CAT,Catalina Airlines Inc.",
    "CAV,Air Virginia",
    "CAZ,CAT Aviation",
    "CB,Air One Inc.",
    "CBA,Carriba Air Inc.",
    "CC,Air Atlanta Icelandic",
    "CC (1),Air Atlanta Inc.",
    "CCA,Cityhop",
    "CCO,Chester County Aviation",
    "CDQ,Kitty Hawk International",
    "CEN,Cen-Tex Airlines Inc.",
    "CEQ,All Canada Express",
    "CET,Century Airlines (Cal)",
    "CF,Compan. De Aviacion Faucett",
    "CGA,Chicago Airlines",
    "CGL,Viking International Airlines Inc.",
    "CGO,Chicago Air Taxi",
    "CH,Bemidji Airlines",
    "CHD,Chandler Flyer",
    "CHE,Charter Airlines",
    "CHF,Hammonds Commuter Air Serv.",
    "CHI,Chitina Air Service",
    "CHL,Channel Flying Inc.",
    "CHO,Choi Aviation Inc.",
    "CHS,Christman Air System",
    "CHT,Charter One",
    "CI,China Airlines Ltd.",
    "CIQ,A/S Conair",
    "CIS,Cal Sierra Airlines",
    "CJ,Colgan Airways Inc.",
    "CK,China Cargo Airline",
    "CKI,Inlet Airlines",
    "CL,Capitol International Inc.",
    "CLB,Columbia Pacific Airlines",
    "CLD,Hawaii Pacific Helicopters",
    "CLF,California Air Shuttle",
    "CLQ,Aero Transcolombiana",
    "CM,Compania Panamena (Copa)",
    "CMB,Combs Airways Inc.",
    "CMD,Command Airways Inc.",
    "CMQ,Servicio De Carga Aerea Sa",
    "CMR,Commuter Airlines (Ca)",
    "CNN,Centennial Airlines",
    "CO,Continental Air Lines Inc.",
    "COB,Cotton Belt Aviation Inc.",
    "COC,Cochise Airlines Inc.",
    "COK,Coker Aviation Inc.",
    "COL,Presidential Express",
    "COM,Commutair",
    "COR,Coral Air Inc.",
    "COS,Cosmopolitan Comm. Airline",
    "COT,Southeastern Commuter",
    "COU,Courtesy Air Service Inc.",
    "CP,Compass Airlines",
    "CP (1),Canadian Airlines International Ltd.",
    "CPA,Capitol Air Lines",
    "CPC,Coastal Plains Commuter Inc.",
    "CPQ,Capitol Air Express Inc.",
    "CPT,Capitol Air",
    "CQI,Conquest Airline Corp.",
    "CRA,Commuter Airlines",
    "CRB,Caribair",
    "CRE,Caribbean Express Inc.",
    "CRH,Cresent Medivac",
    "CRN,Crown Aviaiton Inc.",
    "CRO,Crown Airways Inc.",
    "CRP,Corporate Air",
    "CRT,Crested Butte Air Service",
    "CRV,Acropolis Aviation Ltd.",
    "CS,Continental Micronesia",
    "CSA,Casino Airlines",
    "CSK,Catskill Airways Inc.",
    "CSL,Central States Air Lines",
    "CSM,Chisum Flying Service",
    "CSN,Casino Airlines (1)",
    "CSQ,Celtic Tech Jet Ltd.",
    "CTA,Century Airlines",
    "CTH,Cothron Aviation",
    "CTL,Coastal Air Transport",
    "CTQ,Custom Air Transport",
    "CTR,Chartair",
    "CUQ,Caicos Caribbean Airways",
    "CUS,Custom Air Service",
    "CV,Cargolux Airlines International S.A",
    "CVA,Catalina-Vegas Airlines",
    "CVQ,Caraven S.A.",
    "CW,Air Marshall Islands Inc.",
    "CWQ,Canadian Aviation Express",
    "CX,Cathay Pacific Airways Ltd.",
    "CYQ,Corse Air International",
    "CZ,China Southern Airlines",
    "CZ (1),Cascade Airways Inc.",
    "CZQ,Aviation Concepts",
    "D7,Air Asia X Berhad dba Airasia X",
    "D8,Norwegian Air International Ltd.",
    "DA,Dan Air Services",
    "DAN,Danbury Airlines",
    "DAU,Dauphin Island Airways",
    "DC,Caribbean Air Cargo Co Ltd.",
    "DE,Condor Flugdienst",
    "DEA,Downeast Airlines Inc.",
    "DEC,Decatur Aviation",
    "DFS,Downeast Flying Service",
    "DH,Independence Air",
    "DH (1),Discovery Airways Inc.",
    "DHL,DHL Airlines Inc.",
    "DHQ,DHL Aero Expresso",
    "DI,Norwegian Air UK Limited",
    "DIA,Drummond Island Air",
    "DIC,Dickman Aviation Services",
    "DIR,Northern Star",
    "DL,Delta Air Lines Inc.",
    "DM,Maersk Air I/S",
    "DO,Compania Dominicana De Avia",
    "DOL,Dolphin Airways Inc.",
    "DOQ,Aerolineas Dominicanas S A",
    "DPA,Desert Pacific Airways",
    "DPI,Desert Pacific Airlines Inc.",
    "DQQ,Caribwest Airways Ltd.",
    "DSA,Desert Sun Airlines",
    "DST,Desert Airlines",
    "DUL,Dulles Express",
    "DUZ,Summit Airlines Inc.",
    "DVO,Devoe Airlines",
    "DWN,Dawn Air",
    "DXA,Dolphin Express",
    "DY,Norwegian Air Shuttle ASA",
    "E0,EOS Airlines, Inc.",
    "E7,Estafeta Carga Aereas",
    "E72,European Aviation Air Charter",
    "E8,USAfrica Airways Inc.",
    "E9,Evelop Airlines SL",
    "E9 (1),Boston-Maine Airways",
    "EA,Eastern Air Lines Inc.",
    "EAI,Executive Aviation Inc.",
    "EAQ,Executive Airlines, SL",
    "EAS,Eastern Caribbean Airways",
    "EAT,Eastman Airways Inc.",
    "EBQ,Execaire Aviation Ltd.",
    "ECA,Eagle Airlines (1)",
    "ECI,Eastern Carolina Aviation",
    "ECL,Eclipse Airlines",
    "ECQ,Trans American Charter",
    "ECR,East Coast Airways",
    "ED,Andes",
    "EDE,Ede Aire",
    "EE,Eastern Airlines Group Inc.",
    "EET,Exec Express II",
    "EEX,Exec Express",
    "EEZ,Eagle Aviation Inc.",
    "EFA,Executive Four Winds",
    "EFQ,Trans Europ. Airways France",
    "EGA,Eagle Airline",
    "EGJ,Eagle Jet Charter",
    "EGL,Eagle Commuter Airlines Inc.",
    "EH,Saeta Airlines",
    "EHA,East Hampton Aire Inc.",
    "EI,Aer Lingus Plc",
    "EIA,East. Indiana Aviation Sale",
    "EJQ,Aeroejecutivos C.A.",
    "EK,Emirates",
    "EKA,Air Pacific Inc.",
    "ELL,Ellis Air Taxi Inc.",
    "EM,Empire Airlines Inc.",
    "EMA,Emerald Airlines",
    "EME,Metro Express",
    "EMP,Empire Airlines Inc. (2)",
    "ENT,Enterprise Airlines Inc.",
    "EOQ,Aeroservicios Ecuatorianos",
    "EP,All Star Airlines Inc.",
    "EQ,Tame Linea Aerea del Ecuador Tme EP",
    "ER,Astar USA, LLC",
    "ERI,Erie Airways Inc.",
    "ERO,Empire Aero Service",
    "ERQ,Aero Trades (Western) Ltd.",
    "ESQ,Seagreen Air Transport",
    "ET,Ethiopian Airlines",
    "EU,Ecuatoriana De Aviacion",
    "EUQ,Air Atlanta Europe",
    "EV,ExpressJet Airlines Inc.",
    "EVA,Evanston Aviation",
    "EW,Eurowings GmbH",
    "EWA,Rio Grande Air",
    "EXA,Columbia Pacific Airlines (1)",
    "EXC,Hapag-Lloyd Executive GmbH",
    "EXC (1),Executive Charter",
    "EXP,Express Airlines II",
    "EXQ,Export Air Del Peru S.A.",
    "EXR,Executive Transport",
    "EY,Etihad Airways",
    "EZ,Evergreen International Inc.",
    "F2,Omega Air Holdings d/b/a Focus Air",
    "F8,Freedom Airlines d/b/a HP Expr",
    "F9,Frontier Airlines Inc.",
    "FAA,Falwell Aviation Inc.",
    "FAC,Falcon Airways Inc.",
    "FAQ,Fast Air Carrier Ltd. (1)",
    "FAR,Federal Armored Service Inc.",
    "FB,Translift Airways Limited",
    "FBI,Young Flying Service",
    "FC,Fast Colombia SAS d/b/a Viva Colombia",
    "FCA,Southern Airlines Inc.",
    "FCQ,Falcon Air Express",
    "FD,Air Puerto Rico Airlines",
    "FDA,Florida Airlines Inc.",
    "FDM,Freedom Airlines Inc.",
    "FDQ,Great American Airways",
    "FE,Primaris Airlines Inc.",
    "FF,Tower Air Inc.",
    "FFQ,Five Star Airlines",
    "FH,Frontier Horizon Inc.",
    "FI,Icelandair",
    "FJ,Air Pacific Ltd.",
    "FJ (1),Air Pacific Ltd. (1)",
    "FL,AirTran Airways Corporation",
    "FL (1),Frontier Airlines Inc. (1)",
    "FLA,Florida Air",
    "FLM,Flamenco Airways Inc.",
    "FLR,Flirite Inc.",
    "FNQ,Fine Airlines Inc.",
    "FOR,Ford-Aire Inc.",
    "FOS,Foster Aviation",
    "FP,TAG Aviation S.A.",
    "FQ,Air Aruba",
    "FRA,Freedom Air",
    "FRS,Four Star Aviation",
    "FRT,Frontier Commuter",
    "FRW,Farwest Airlines",
    "FS,Serv De Trans Aereos Fuegui",
    "FSA,F.S. Air Service",
    "FT,Flying Tiger Line Inc.",
    "FTI,Southern Airways",
    "FVA,Skycraft Inc.",
    "FW,Skybus Inc.",
    "FWA,Far West Airlines",
    "FX,Federal Express Corporation",
    "FYB,Flying Boat",
    "FYQ,Flying Group Lux S.A.",
    "G3,VRG Linhas Aereas S A",
    "G3 (1),Aerochago S.A.",
    "G4,Allegiant Air",
    "G7,GoJet Airlines LLC d/b/a United Express",
    "GA,P.T. Garuda Indonesian Arwy",
    "GAL,Galena Air Service",
    "GAQ,Galaxy Airlines Inc.",
    "GAS,General Aviation Inc.",
    "GAT,Gulf Air Taxi",
    "GAV,Gromer Aviation Inc.",
    "GBA,Central States Airline",
    "GBQ,G5 Executive Ag",
    "GCA,Golden Carriage Aire",
    "GCH,Grand Canyon Helicopters",
    "GCQ,Glock Aviation GmbH",
    "GCQ (1),General Air Cargo",
    "GCS,Midway Commuter",
    "GD,Transp. Aereos Ejecutivos",
    "GDW,Golden West Airlines Inc.",
    "GE,Lufthansa Cargo Airlines",
    "GF,Gulf Air Company",
    "GF (1),Gulf Air Transport Inc.",
    "GFC,Gulf Flite Center",
    "GFQ,Gulf And Caribbean Cargo",
    "GG,Cargo 360, Inc.",
    "GG (1),Tropical Airways Inc.",
    "GG (2),Grenada Airways Ltd.",
    "GH,Ghana Airways Corporation",
    "GHA,Green Hills Aviation Ltd.",
    "GIQ,Global International Airways Corp.",
    "GJ,Eurofly S.p.A",
    "GJ (1),Mexicargo",
    "GKQ,Laker Airways",
    "GL,Miami Air International",
    "GLW,Golden West Airlines Co.",
    "GM,Air America",
    "GMA,Green Mountain Airlines Ltd.",
    "GOF,Copper State Airlines",
    "GPA,GP Express Airlines",
    "GQ,Big Sky Airlines Inc.",
    "GR,Gemini Air Cargo Airways",
    "GRA,Gorda Aero Service",
    "GRD,Grand Canyon Airlines",
    "GRO,Isle Royale Seaplane Serv.",
    "GS,Air Foyle Limited",
    "GSA,Golden Gate Airlines Inc.",
    "GSC,GCS Air Service",
    "GSL,Golden South Airlines Inc.",
    "GSQ,Global Supply System",
    "GST,Golden State Airlines",
    "GU,Aviateca",
    "GUL,Gull Air Inc.",
    "GUN,Gunnel Aviation Inc.",
    "GV,Grant Aviation",
    "GW,SkyGreece Airlines S A",
    "GW (1),Central American Airlines",
    "GWA,Great Western Aviation",
    "GWE,Ross Aviation Inc.",
    "GX,Air Ontario Limited",
    "GY,Guyana Airways Corporation",
    "H2,City Bird",
    "H5,Magadan Airlines",
    "H6,Hageland Aviation Service",
    "HA,Hawaiian Airlines Inc.",
    "HAQ,Hapag Lloyd Flug.",
    "HAR,Friendship Air Alaska",
    "HB,Homer Air",
    "HB (1),Air Club International",
    "HBQ,Harris Air Services",
    "HCQ,Av Atlantic",
    "HEL,Helitrans",
    "HEP,Hub Express",
    "HER,Markair Express",
    "HET,Helitac Aviation",
    "HEU,Heussler Air Service Corp",
    "HFQ,Haiti Air Freight Intl",
    "HFS,Hensley Flying Service Inc.",
    "HIQ,Business Flights Ltd.",
    "HJ,Air Haiti S. A.",
    "HK,Time Air Sweden Ab",
    "HLQ,Heavylift Cargo Airlines Lt",
    "HMZ,Long Island Airlines",
    "HNS,Haines Airways",
    "HOL,Holiday Airlines Inc.",
    "HOQ,Holidair Airways",
    "HOR,Iowa Airlines",
    "HP,America West Airlines Inc. (Merged with US Airways 9/05. Stopped reporting 10/07.)",
    "HPZ,Pacific Coast Airlines",
    "HQ,Harmony Airways",
    "HQ (1),Business Express",
    "HQQ,Harlequin Air Corporation",
    "HRA,Trans-National Airlines Haw",
    "HRG,Heritage Airlines",
    "HRQ,Tar Airlines",
    "HRZ,Allegheny Airlines",
    "HSV,Harbor Air Service",
    "HSZ,Scenic Airlines Inc.",
    "HU,Hainan Airlines Company Limited",
    "HUB,Skystream Airlines",
    "HVQ,Transavia Holland B. V.",
    "HWK,Hawking Corporation",
    "HX,Hong Kong Airlines Limited",
    "HX (1),Guy-America Airways Inc.",
    "HY,Uzbekistan Airways",
    "HYA,Hyannis Aviation",
    "I4,Scott Air LLC dba Island Air Express",
    "IA,Iraqi Airways",
    "IB,Iberia Air Lines Of Spain",
    "ICF,Iowa City Flying Service",
    "ICH,Island Air Charters",
    "ICQ,Iscargo Hf",
    "IDQ,Intl Air Service Co. Ltd.",
    "IEX,Island Air Express",
    "IG,Airitaly S.P.A.",
    "IGA,Global Jet Austria, GmbH",
    "III,Idee Industries Inc.",
    "IJ,Great Wall Airlines Company Limited",
    "IKQ,Tradewinds Airways Ltd.",
    "IMP,Imperial Airlines Inc.",
    "IN,Aerlinte Eireann Teoranta",
    "INA,Inland Empire Airlines Inc.",
    "IND,Indiana Airways Inc. (Pa)",
    "INQ,Independent Air Inc.",
    "IOW,Iowa Airways",
    "IPI,Indo-Pacific International",
    "IPQ,Aruba Cargo",
    "IPS,Island Express",
    "IR,Iran National Airlines Corp",
    "IRQ,Interstate Airlines Inc.",
    "ISL,Island Airlines",
    "ITQ,Interamericana De Aviacion",
    "IW,Air Liberte Aka Aom Minerve",
    "IW (1),Best Airlines Inc.",
    "IXQ,Inair",
    "J2,Azerbaijan Airlines CJSC",
    "J5,Kalinin Aviation LLC d/b/a Alaska Seaplanes",
    "J6,Larrys Flying Service",
    "J7,Valujet Airlines Inc.",
    "JAG,JetAlliance Flugbetriebs d/b/a JAF Airservice",
    "JAM,Sunbelt Airlines Inc.",
    "JAQ,Jamaica Air Freighters",
    "JB,Helijet International Inc.",
    "JCM,Air Chico",
    "JCQ,Jet Connection Businessflight Ag",
    "JCZ,Rocky Mountain Airways Inc.",
    "JD,Beijing Capital Airlines",
    "JD (1),Japan Air System Co. Ltd.",
    "JER,Jer-Don Air",
    "JF,L.A.B. Flying Service Inc.",
    "JI,Jade Cargo International",
    "JI (1),Midway Airlines Inc.",
    "JIQ,Jet Fleet Intl. Airlines",
    "JJ,Transportes Aeros Meridiona",
    "JK,Spanair S.A.",
    "JK (1),Sunworld International Airways Inc.",
    "JKQ,Express One International Inc.",
    "JL,Japan Air Lines Co. Ltd.",
    "JM,Air Jamaica Limited",
    "JMA,Jim Air Inc.",
    "JN,Excel Airways",
    "JO,Jalways Co. Ltd.",
    "JQ,Jetstar Airways Pty Limited",
    "JR,Aero California",
    "JTA,Jetaire Airlines Inc.",
    "JU,Air Serbia",
    "JU (1),Yugoslav Airlines",
    "JW,Arrow Air Inc.",
    "JX,Southeast Airlines",
    "JX (1),Jes Air Ltd.",
    "JXQ,Allarco Developments Ltd.",
    "JZ,Japan Air Charter Co. Ltd.",
    "K2,Paklook Air, Inc d/b/a/ Airlift Alaska d/b/a/ Yute Commuter Service",
    "K3,Venture Travel LLC d/b/a Taquan Air Service",
    "K5,Seaport Airlines, Inc.",
    "K7,Arizona Express",
    "K8,Dutch Caribbean Airlines",
    "KA,Hong Kong Dragonair",
    "KAH,Kenmore Air Harbor",
    "KAI,Knight Airlines Inc.",
    "KAQ,Kalitta Air LLC",
    "KAT,Katmai Air",
    "KB,Haiti Air",
    "KD,Western Global",
    "KE,Korean Air Lines Co. Ltd.",
    "KEA,Keene Airways",
    "KEE,Key Airlines",
    "KET,Ketchikan Air Service",
    "KFS,King Flying Service",
    "KH,Aloha Air Cargo",
    "KI,Time Air Ltd.",
    "KIN,Golden Pacific Airlines",
    "KKQ,Transmeridian Air Cargo Ltd.",
    "KL,Klm Royal Dutch Airlines",
    "KLQ,Kalitta Charters II",
    "KM,Air Malta",
    "KN,Morris Air Corporation",
    "KO,Alaska Central Express",
    "KOD,Kodiak Airways Inc.",
    "KP,Kiwi International",
    "KQ,Kenya Airways PLC",
    "KR,Kitty Hawk Aircargo",
    "KR (1),Kar-Air Oy",
    "KS,Peninsula Airways Inc.",
    "KTQ,Turks Air Ltd.",
    "KU,Kuwait Airways Corp.",
    "KV,Sky Regional Airlines Inc.",
    "KW,Carnival Air Lines Inc.",
    "KWQ,Kelowna Flightcraft Air Cha",
    "KWZ,Crown Air",
    "KX,Cayman Airways Limited",
    "KXZ,Chaparral Airlines Inc.",
    "KZ,Nippon Cargo Airlines",
    "L2,Lynden Air Cargo Airlines",
    "L3,Lynx Aviation d/b/a Frontier Airlines",
    "L7,Lan Colombia",
    "LA,Lan-Chile Airlines",
    "LAA,La Skycab Airline",
    "LAH,La Helicopter",
    "LAN,Lancer Aviation Inc.",
    "LAP,Lapsa",
    "LAR,Laredo Air",
    "LAS,Lake Union Air Service",
    "LAW,Lawrence Aviation",
    "LB,Lloyd Aereo Boliviano S. A.",
    "LBQ,Millon Air Inc.",
    "LC,Varig Logistica S/A",
    "LC (1),Lineas Aereas Del Caribe",
    "LC (2),Legend Airlines",
    "LCQ,Latcr",
    "LCT,Lake Coastal",
    "LDM,Lauda Motion GmbH",
    "LEQ,Le Point Air",
    "LEX,L'Express",
    "LF,CFM Inc d/b/a Contour Airlines d/b/a One Jet Shuttle",
    "LF (1),Jettrain Airlines",
    "LG,Luxair",
    "LGQ,Lineas Aereas Allegro",
    "LH,Lufthansa German Airlines",
    "LHA,Havasu Airlines",
    "LIQ,Liat (1974) Limited",
    "LJ,Jin Air Co Ltd.",
    "LKN,Loken Aviation",
    "LLA,Lakeland Aviation",
    "LNQ,Aerolineas Latinas",
    "LO,Polskie Linie Lotnicze",
    "LOQ,Lorair Ltd.",
    "LOS,Aero Commuter",
    "LP,Lan Peru Airlines",
    "LP (1),The Hawaii Express Inc.",
    "LR,Lacsa",
    "LRQ,Air Liberte",
    "LS,Jet2.com",
    "LSQ,Lineas Aereas Suramerican",
    "LSZ,Alaska Island Air Inc.",
    "LT,Luftransport-Unternehmen",
    "LU,Air Atlantic Dominicana",
    "LUQ,Lineas Aereas La-Tur Sa",
    "LVA,Las Vegas Airlines",
    "LW,Pacific Wings Airlines",
    "LX,Swiss International Airlines",
    "LXQ,Translux Intl Airlines",
    "LY,El Al Israel Airlines Ltd.",
    "LZ,Balkan Bulgarian Airlines",
    "LZQ,Leisure International Airways Ltd.",
    "M3,ABSA-Aerolinhas Brasileiras",
    "M6,Amerijet International",
    "M7,Aerotransportes Mas De Crga",
    "MA,Malev Hungarian Airlines",
    "MAA,Maui Airlines",
    "MAC,Mackey International Inc.",
    "MAE,Mid-Atlantic Exp. Airlines",
    "MAG,Magum Airlines",
    "MAI,The Maui Commuter",
    "MAQ,Millardair Limited",
    "MAR,Marshall's Air Inc.",
    "MAV,Maverick Airways",
    "MAX,Maxair Inc.",
    "MB,Samoa Airlines Inc.",
    "MC,Transtar Airlines Corp.",
    "MCO,Mid Continental Airlines In",
    "MDC,Midcontinent Airlines",
    "MDU,Metro Airlines",
    "ME,Middle East Airlines S.A.L.",
    "MEA,Manhattan Express",
    "MET,Metroplex Airlines",
    "MEX,Metro Express 11",
    "MF,Xiamen Airlines Co., Ltd.",
    "MG,Champion Air",
    "MGQ,Montana Austria Flugetrieb",
    "MH,Malaysian Airline System",
    "MHK,Mohawk Airlines",
    "MHO,Mahalo Air",
    "MIN,Minuteman Aviation Inc.",
    "MIS,Midstate Airlines Inc.",
    "MIY,Midway Aviation Inc.",
    "MJ,Lapa-Lineas Aereas Privadas",
    "MKQ,Airmark Corporation",
    "ML,Aero Costa Rica",
    "ML (1),Midway Airlines Inc. (1)",
    "MLL,Miller Air Transportersinc",
    "MLX,Merlin Express",
    "MM,Euro Atlantic Airways",
    "MM (1),Sociedad Aeronautica De Med",
    "MMH,Ocean Airways Inc.",
    "MNA,M & N Aviation",
    "MNZ,Cumberland Airlines",
    "MOQ,Morningstar Air Express",
    "MP,Martinair Holland N.V.",
    "MPA,Mid Pacific Airlines Inc.",
    "MPC,Mid Pacific Air Corp.",
    "MQ,Envoy Air",
    "MRC,Marco Island Airways Inc.",
    "MRR,Merrimack Aviation Inc.",
    "MS,Egyptair",
    "MST,Midwest Aviation",
    "MT,Thomas Cook Airlines Uk Ltd.",
    "MT (1),Flying Colours Airlines Ltd.",
    "MTA,Arkansas Traveler Airline",
    "MTR,Metroflight Airlines",
    "MTV,Mst Aviation",
    "MTX,Mountain Air Express",
    "MU,China Eastern Airlines",
    "MU (1),Mcclain Airlines Inc.",
    "MUA,Manu'a Air Transport",
    "MUQ,Aerolineas Mundo",
    "MVA,Mississippi Valley Airlines",
    "MW,Mokulele Flight Services, Inc.",
    "MWA,Mountain West Airlines",
    "MX,Compania Mexicana De Aviaci",
    "MY,MAXjet",
    "MYQ,Lineas Aereas Mayas (Lamsa)",
    "MZ,P.N. Merpati Nusantara Air.",
    "N5,Skagway Air Service",
    "N5 (1),Nations Air Express Inc.",
    "N6,Aero Continente",
    "N7,National Airlines",
    "N8,National Air Cargo Group Inc d/ba National Airlines",
    "NA,North American Airlines",
    "NAC,National Air Commuter Inc.",
    "NAP,Npa Inc.",
    "NAQ,Nationair",
    "NAS,Nasville Eagle",
    "NAT,National Air",
    "NB,Sterling Cargo",
    "NBQ,Sterling Airways A/S",
    "NC,Northern Air Cargo Inc.",
    "NCA,National Commuter Airlines",
    "NCE,Northcoast Executive",
    "ND,Intair",
    "NEB,New Born Wings Inc.",
    "NEC,Nor East Commuter Airlines",
    "NEQ,North East Cargo Airlines",
    "NER,Northeast Express Regional (1)",
    "NET,Nacnet",
    "NEV,Nevada Airlines",
    "NEW,New England Airlines Inc.",
    "NEX,Northeast Express Regional",
    "NFA,National Florida Airlines",
    "NFQ,North Cariboo Flying Serv.",
    "NG,Lauda Air Luftfahrt Ag",
    "NGL,Northern Eagle Airways Inc.",
    "NH,All Nippon Airways Co.",
    "NHA,Newair Flight Inc.",
    "NI,Lanica",
    "NI (1),American International Airways Inc.",
    "NJ,Vanguard Airlines Inc.",
    "NJA,N. J. Merrimack Airlines",
    "NJE,NetJets Transportes Aereos, S.A.",
    "NK,Spirit Air Lines",
    "NKQ,Norcanair",
    "NLQ,Nolinor Aviation",
    "NN,Northward Airlines Ltd.",
    "NOQ,Novair Airlines Ab",
    "NPA,North Pacific Airlines Inc.",
    "NPT,E. J. A. Newport Inc.",
    "NQ,Air Japan Co",
    "NRN,Northern Airlines Inc.",
    "NSQ,North Coast Air Services",
    "NTH,Northaire",
    "NTQ,National Aviation Consult.",
    "NUM,Lake State Airways",
    "NUQ,Air Niugini",
    "NVQ,Novair International Airways Ltd.",
    "NVX,Nevis Express",
    "NW,Northwest Airlines Inc.",
    "NWQ,N. W. Territorial Airways",
    "NWS,Northwest Seaplanes",
    "NXQ,North Vancouver Air",
    "NY,New York Air Inc.",
    "NYH,New York Helicopter Corp.",
    "NZ,Air New Zealand",
    "O6,Oceanair Linhas Aereas S A",
    "OA,Olympic Airways",
    "OAK,Harbor Airlines Inc.",
    "OB,Oasis International Airlines S.A.",
    "OC,Air California Inc.",
    "OCR,Ocean Reef Airways Inc.",
    "OD,Emerald Airlines Inc.",
    "OD (1),Aerocondor",
    "OE,Westair Airlines Inc.",
    "OF,Air Finland Ltd.",
    "OFF,Offshore Logistics",
    "OH,PSA Airlines Inc.",
    "OH (1),Comair Inc.",
    "OHA,Omaha Aviation",
    "OHZ,Caribbean Air Services Inc.",
    "OI,Prestige Airways",
    "OIZ,Corporate Air Inc.",
    "OJ,Fly Jamaica Airways Limited",
    "OK,Czech Airlines",
    "OMH,Omniflight Helicopters",
    "OMI,Omniflight Airways",
    "OMK,Omak Aviation Airlines",
    "OMN,Omni Pacific Airline",
    "OMQ,Quassar De Mexico",
    "ON,Air Nauru",
    "ONQ,Nunasi-Central Airlines Ltd.",
    "OO,SkyWest Airlines Inc.",
    "OOZ,Sun Aire Lines",
    "OP,Flying Boat Inc.",
    "OP (1),Air Panama Internacional",
    "OPQ,Orion Air",
    "OR,TUI Airlines Nederland B.V.",
    "ORI,Orion Air Inc.",
    "OS,Austrian Airlines",
    "OTT,Otter Air",
    "OV,Aeromaritime",
    "OW,Executive Airlines",
    "OWQ,Ontario Worldair Limited",
    "OWS,Air Kentucky Air Lines",
    "OZ,Asiana Airlines Inc.",
    "OZ (1),Ozark Air Lines Inc. (1)",
    "OZA,Ozark Air Lines Inc.",
    "P3,Cargologicair Limited",
    "P9,Pro Air Inc.",
    "PA,Florida Coastal Airlines",
    "PA (1),Pan American World Airways (1)",
    "PA (2),Pan American World Airways",
    "PAI,Permian Airways Inc.",
    "PAQ,Patriot Airlines Inc.",
    "PAT,Panorama Air Tours",
    "PBA,Provincetown Boston",
    "PBQ,Privatair",
    "PCA,Piasa Commuter Airlines",
    "PCL,Pacific Cal Air",
    "PCQ,Pace Airlines",
    "PCR,Pacair",
    "PD,Porter Airlines, Inc.",
    "PD (1),Pem-Air Limited",
    "PDA,Trans Southern Airways",
    "PDQ,Antigua Paradise Airways",
    "PE,Air Europe S.P.A.",
    "PE (1),People Express Airlines Inc.",
    "PEA,Pearson Aircraft",
    "PEQ,SkyLink Express Inc.",
    "PER,Air Pennsylvania",
    "PET,Petro Aviation Corp.",
    "PF,Primera Air Scandinavia",
    "PFA,Pacificair",
    "PFQ,Asia Pacific",
    "PH,Polynesian Airlines Ltd.",
    "PHL,Phillips Airlines",
    "PHR,Pinehurst Airlines Inc.",
    "PHX,Phoenix Airlines",
    "PI,Piedmont Aviation Inc.",
    "PIA,Pacific Island Airways",
    "PIQ,Pacific International Airlines",
    "PK,Pakistan International Airlines",
    "PKQ,Skyworld Airlines Inc.",
    "PL,Aero Peru",
    "PLA,Planes Inc.",
    "PLQ,Planet Airways",
    "PLZ,Polar Airways Inc.",
    "PM,Prescott Support Company, Inc.",
    "PMA,Pacific Missionary Aviation",
    "PMT,Pilgrim Aviation & Airlines",
    "PN,Pan American Airways Corp.",
    "PNM,Panama Aviation",
    "PNQ,Panagra Airways",
    "PNR,Pioneer Airways",
    "PNS,Pine State Airlines",
    "PO,Polar Air Cargo Airways",
    "POA,Potomac Air",
    "POC,Pocono Airlines Inc.",
    "POL,Polar International",
    "POM,Pompano Airways",
    "PON,Ponderosa Aviation Inc.",
    "POQ,Polyot Airlines",
    "PP,Jet Aviation Business Jets AG",
    "PPQ,Propair",
    "PR,Philippine Airlines Inc.",
    "PRA,Princeton Aviation Corp",
    "PRD,Paradise Island Airlines",
    "PRM,Prime Air Inc.",
    "PRN,Puerto Rico Intl Airlines",
    "PRO,Sundance Airlines",
    "PRP,Propheter Aviation",
    "PRQ,Florida West Airlines Inc.",
    "PRR,Pro Air Services",
    "PS,Ukraine International Airlines",
    "PS (1),Pacific Southwest Airlines",
    "PSA,P.S. Air Freight Inc.",
    "PT,Piedmont Airlines",
    "PT (1),Capital Cargo International",
    "PTQ,Flight International",
    "PU,Primeras Lineas Uruguays For International",
    "PUQ,Aeropuma",
    "PV,Eastern Provincial Airways",
    "PVA,Precision Valley Aviation",
    "PW,Pacific Western Airlines",
    "PY,Surinam Airways Limited",
    "PZ,Lineas Aereas Paraguayas",
    "Q2,Zambia Airways Corp.",
    "Q4,Master Top Linhas Aereas, Ltda.",
    "Q5,40-Mile Air",
    "Q7,SkyBahamas Airlines Ltd.",
    "Q7 (1),Sobelair",
    "QB,Inter-Quebec (Intair)",
    "QD,Grand Airways Inc.",
    "QF,Qantas Airways Ltd.",
    "QG,Sky West Aviation Inc.",
    "QH,Air Florida Inc.",
    "QK,Jazz Aviation LP",
    "QN,Royal Air",
    "QO,Aeromexpress",
    "QOZ,Bar Harbor Airways",
    "QQ,Reno Air Inc.",
    "QR,Qatar Airways (Q.C.S.C)",
    "QS,Travel Service, a.s. d/b/a SmartWings",
    "QS (1),Northeastern International Airways",
    "QT,Transportes Aereos Mercantiles Panamericanos S.A",
    "QTQ,Aer Turas Teoranta",
    "QWE,Qwest Commuter",
    "QX,Horizon Air",
    "QY,European Air Transport Leipzig GmbH",
    "R0,Tantalus Air Ltd. d/b/a Harbour Air d/b/a Harbour Air Seaplanes",
    "R2,Orenburg Airlines",
    "R3,Air Company Yakutia",
    "RAL,Royal-Air Ltd.",
    "RAN,Pan Am Express",
    "RAQ,Arista International Airlines Inc.",
    "RAY,Royale Airlines Inc.",
    "RB,Syrian Arab Airlines",
    "RC,Republic Airlines Inc.",
    "RCA,Mid-South Aviation Inc. (1)",
    "RD,Ryan International Airlines",
    "RD (1),Airlift International Inc.",
    "RDR,Rader Aviation",
    "RDW,Redwing Airways",
    "REA,Realwest Airlines",
    "RED,Aerosun International Airlines Inc.",
    "REQ,Renown Aviation",
    "RES,Resort Airlines Inc.",
    "RG,Varig S. A.",
    "RGQ,Regency Air",
    "RHA,Royal Hawaiian Airways Inc.",
    "RIA,Trump Air",
    "RIC,Richardson Aviation",
    "RID,Indiana Airways Inc. (Reid",
    "RIO,Rio Airways Inc.",
    "RIQ,Rich International Airways",
    "RIV,Riverside Air Service",
    "RJ,Alia-(The) Royal Jordanian",
    "RJS,Michigan Airways Inc.",
    "RK,Air Afrique",
    "RL,UltrAir",
    "RL (1),Aerolineas Nicaraguenses",
    "RLQ,Reliant Airlines",
    "RMH,Rocky Mountain Helicopters",
    "RMV,Ram Aviation",
    "RNQ,Mytravel Airways",
    "RO,Tarom Romanian Air Transpor",
    "ROA,Royal West Airways Inc.",
    "ROE,Roederer Aviation Inc.",
    "ROQ,Aero Uruguay (1)",
    "ROS,Great Western Airlines Inc.",
    "ROY,Royal American Airways Inc.",
    "RP,Chautauqua Airlines Inc.",
    "RRT,Air Freight Express",
    "RSC,Resort Commuter",
    "RSQ,Aerotransportes Entre Rios",
    "RSV,Air-Serve",
    "RTQ,Aerotour Dominicano Cxa",
    "RU,AirBridgeCargo Airlines Limited",
    "RV,Air Canada rouge LP",
    "RV (1),Reeve Aleutian Airways Inc.",
    "RVA,Reeves Air",
    "RVQ,Reeve Air Alaska, LLC",
    "RWG,Air Rovers Inc.",
    "RY,Redcoat Air Cargo Limited",
    "RZZ,Aeromech Inc.",
    "S3,Santa Barbara Airlines, C.A",
    "S4,Sata Internacional",
    "S5,Shuttle America Corp.",
    "S6,Star Air A/S",
    "S6 (1),Mountain Bird/Salmon Air",
    "SA,South African Airways",
    "SAC,Sundorph Aeronautical Corp.",
    "SAH,Sea Air Shuttle",
    "SAI,Seair Alaska Airlines Inc.",
    "SAJ,Sajen Air Inc.",
    "SAL,Sallee's Aviation",
    "SAM,Samoan Airlines Inc.",
    "SAQ,Southern Air Transport Inc.",
    "SAS,Air Cortez International",
    "SAX,Air Americana",
    "SBA,Saber Aviation Inc.",
    "SBN,Suburban Airlines",
    "SBQ,Skyjet S.A.",
    "SCN,Scott Aviation",
    "SCO,Seaco Airlines Inc.",
    "SCT,Skyline Airlines",
    "SCY,South Coast Airways",
    "SDQ,St. Lucia Airways Limited",
    "SE,XL Airways France",
    "SE (1),Das Air Cargo",
    "SEB,Seaborne Virgin Islands, Inc.",
    "SEG,Seagull Air Service",
    "SEM,Semo Aviation Inc.",
    "SEP,Southern Express Airways",
    "SEQ,Sky Service F.B.O.",
    "SER,Service Aviation Inc.",
    "SFI,Stahmannfarms Inc.",
    "SFO,Sfo Helicopter Airlines Inc.",
    "SFQ,Pacific International Airlines Inc.",
    "SFQ (1),Southern Frontier Air Trans",
    "SFS,Sierra Flite Service Inc.",
    "SG,Jetsgo",
    "SH,Servicio Aereo De Honduras",
    "SHS,Shasta Air Inc.",
    "SHV,Shavano Air Inc.",
    "SI,Sierra Pacific Airlines",
    "SI (1),Jet America Airlines Inc.",
    "SIA,Silver State Airlines",
    "SIQ,Premiair",
    "SIQ (1),Scanair",
    "SJA,San Juan Airlines Inc.",
    "SJQ,Skycharter (Malton) Ltd.",
    "SK,Scandinavian Airlines Sys.",
    "SKI,Sunbird Inc.",
    "SKJ,Sky-Jet",
    "SKL,Bas Airlines",
    "SKM,Scheduled Skyways",
    "SKS,Silver Kris Services Ltd.",
    "SKT,Skytrain",
    "SLO,Slocum Air Inc.",
    "SLQ,Sky King Inc.",
    "SLR,Salair",
    "SLZ,Southeast Airlines Inc.",
    "SM,Sunworld International Airlines",
    "SMC,Stopher Mechanical Inc.",
    "SMO,Samoa Aviation Inc.",
    "SMQ,Serv. Aerolineas Mexicanas",
    "SN,Brussels Airlines N.V.",
    "SN (1),Sabena Belgian World Air.",
    "SNA,Aviation Associates",
    "SNB,Ccair",
    "SNC,Gelco Courier Services Inc.",
    "SNE,Shawnee Airlines",
    "SNK,Spernak Airways Inc.",
    "SNV,Sierra Nevada Airways",
    "SO,Swiss World Airways",
    "SOA,Mall Airways Inc.",
    "SOE,Sky One Express",
    "SOL,Soonair Lines Inc.",
    "SOP,South Pacific Island Airway",
    "SPF,Springfield Air Service",
    "SPG,Springdale Air",
    "SPH,Spirit Helicopter",
    "SPQ,Sun Pacific International",
    "SPR,Sportsman Flying Service",
    "SQ,Singapore Airlines Ltd.",
    "SR,Swissair Transport Co. Ltd.",
    "SRA,Star Aviation",
    "SRL,Sterling Air Service",
    "SS,Corsair International",
    "SSA,S. S. Airways Inc.",
    "SSP,Southern Seaplane",
    "SSQ,Superior Airways Limited",
    "SSS,Sierra Express Inc.",
    "SST,Seaplane Shuttle Transport",
    "ST,Belize Airways Limited",
    "STA,State Airlines Inc.",
    "STF,Trans East International",
    "STG,Sedalia Marshall Boonvl Stg",
    "STQ,Skycraft Air Transport Inc.",
    "STR,Star Aviation Corporation",
    "STW,Stateswest Airlines",
    "STZ,Air America Inc.",
    "SU,Aeroflot Russian Airlines",
    "SUA,Sunshine Airlines",
    "SUI,Sun International Airways",
    "SUT,Sunbelt",
    "SUX,Sunair Express LLC",
    "SV,Saudi Arabian Airlines Corp",
    "SVL,Sun Valley Airlines",
    "SVW,Global Jet Luxembourg, Sa",
    "SWA,Sunwest Airlines",
    "SWT,Swift Aire Lines Inc.",
    "SX,Skybus Airlines, Inc.",
    "SX (1),Aeroejecutivo S.A.",
    "SXP,Sierra Expressway",
    "SY,Sun Country Airlines d/b/a MN Airlines",
    "SYB,Skybus Express Airlines",
    "SYM,Skymaster",
    "SZQ,Aerolineas El Salvador S A",
    "T3,Tristar Airlines Inc.",
    "T9,TransMeridian Airlines",
    "TA,Taca International Airlines",
    "TAC,Air Colorado",
    "TAF,Tri Air Freight",
    "TAL,Tejas Airlines Inc.",
    "TAQ,Taquan Air Service",
    "TAS,Yosemite Airlines Inc.",
    "TAT,Talarik Creek Air Taxi",
    "TB,TUI Airlines Belgium N.V. d/b/a Jetairfly",
    "TB (1),USAir Shuttle",
    "TBQ,Triair Bermuda Flt Operations",
    "TCA,Trans Catalina Airlines",
    "TCE,Oceanair Line",
    "TCI,Trans California Airlines",
    "TCL,Trans-Central Airlines Inc.",
    "TCO,Trans-Colorado Airlines",
    "TCQ,Express.Net Airlines",
    "TDQ,Venexcargo",
    "TDQ (1),Transcarga",
    "TEN,Tennessee Airways Inc.",
    "TER,Terre Haute Air Commuter",
    "TFQ,Trans International Airlines Inc.",
    "TFZ,Tri-State Airlines Inc.",
    "TG,Thai Airways International Ltd.",
    "TGO,Trans Air Cargo Inc.",
    "TGQ,Tranps Aereos Guatemaltecos",
    "TH,Transmile Air Service",
    "THA,Tropical Helicopter Airways",
    "THO,Thorson Aviation",
    "THQ,T-Bird Air Inc.",
    "TIA,Trans Air Inc.",
    "TIE,Trans International Express",
    "TIQ,Tyrolean Jet Service",
    "TIS,Skyfreight",
    "TJ,Jet Charter Service Inc.",
    "TK,Turk Hava Yollari A.O.",
    "TKQ,Trans-Air-Link Corporation",
    "TLQ,Trans-Mediterranean Airways",
    "TMA,Trans Mo Airlines",
    "TN,Air Tahiti Nui",
    "TNQ,Emery Worldwide Airlines",
    "TNS,Transtate Aviation",
    "TP,Tap-Portuguese Airlines",
    "TPQ,Aerial Transit Company",
    "TQQ,Taca Peru",
    "TR,Scoot Tigerair Pte Ltd d/b/a Scoot",
    "TR (1),Transbrasil S.A.",
    "TRA,Aeromexico Travel",
    "TRI,Great Plains Airlines Inc.",
    "TRO,Air Molakai",
    "TRP,Trans Pennsylvania Airlines",
    "TRQ,Blue Scandinavia Ab",
    "TRR,Territorial Airlines",
    "TS,Air Transat",
    "TSF,American Central Airlines",
    "TSI,Trans Sierra Airlines Inc.",
    "TSQ,TPI International Airways",
    "TTM,The Time Machine",
    "TUL,Tulsair Beechcraft Inc.",
    "TUR,Turner Aire",
    "TV,Haiti Trans Air",
    "TV (1),Transamerica Airlines Inc.",
    "TVL,Travelair",
    "TW,Trans World Airways LLC",
    "TWA,Trans Western Airlines Utah",
    "TWE,Trans World Express",
    "TX,Soc.Nouvelle Air Guadeloupe",
    "TX (1),Transportes Aereos Nacional",
    "TXA,Texas National Airlines",
    "TXE,Trans Air (Hawaii)",
    "TXQ,British Caribbean Airways",
    "TXY,Texas Airways",
    "TYE,Temsco Airlines",
    "TZ,ATA Airlines d/b/a ATA",
    "TZQ,First Choice Airways",
    "U2,UFS Inc.",
    "U5,USA 3000 Airlines",
    "U7,USA Jet Airlines Inc.",
    "UA,United Air Lines Inc.",
    "UAI,Universal Airways Inc.",
    "UC,Ladeco",
    "UD,Fast Air Carrier Ltd.",
    "UH,US Helicopter Corporation",
    "UMQ,Air Manila Inc.",
    "UN,Transaero Airlines",
    "UNQ,Overseas National Airways",
    "UO,Hong Kong Express Airways Limited",
    "UP,Bahamasair Holding Limited",
    "UR,Empire Airlines Inc. (1)",
    "URQ,Aero Uruguay",
    "US,US Airways Inc.",
    "USA,Air U.S.",
    "UT,Union De Transports Aeriens",
    "UTQ,Universal Airlines Inc.",
    "UX,Air Europa",
    "UX (1),Air Illinois Inc.",
    "UYQ,Aerolineas Uruguayas S.A.",
    "V4,Vensecar International",
    "V5,J.D.Valenciana De Aviation",
    "V8,Iliamna Air Taxi",
    "V9,Eagle Airlines",
    "VA,Virgin Blue International Airlines t/a V Australia",
    "VA (1),Venezuelan International Airways",
    "VAG,Vagabond Aviation Inc.",
    "VAQ,Vacationair",
    "VAS,Valley Air Services",
    "VB,Aeroenlaces Nacionales, S.A. de C.V. d/b/a VivaAerobus",
    "VB (1),Pacific Express",
    "VC,Servicios Avensa",
    "VCQ,Victory Air Transport Inc.",
    "VE,Aerovias Venezolanas-Avensa",
    "VER,Britt Airlines Inc.",
    "VEZ,Valdez Airlines",
    "VH,Aeropostal Alas De Venezuel",
    "VI,Vieques Air Link Inc.",
    "VIG,Viking Express",
    "VIN,Vale International Airlines",
    "VIQ,Volga-Dnepr Airlines",
    "VIQ (1),Viscount Air Service Inc.",
    "VIS,Virgin Islands Seaplane",
    "VJT,VistaJet Limited",
    "VK,Air Tungaru Corporation",
    "VL,Mid-South Aviation Inc.",
    "VLA,Village Aviation",
    "VLY,Valley Airlines",
    "VOQ,Avalon Aviation Ltd.",
    "VP,Viacao Aerea Sao Paulo",
    "VQ,Vintage Props & Jets Inc.",
    "VR,Transportes Aereos De Cabo",
    "VS,Virgin Atlantic Airways",
    "VSA,VistaJet Luftfahrtunternehmen GmbH",
    "VV,Aerosvit Ukranian Airlines",
    "VV (1),Challenge Air Transport Inc.",
    "VV (2),Challenge Air International Inc.",
    "VW,Transportes Aeromar S.A de C.V",
    "VX,Virgin America",
    "VX (1),Aces Airlines",
    "VY,Highland Express Airways",
    "W3,Arik Air International",
    "W4,Mn Aviation, Inc.",
    "W7,Western Pacific Airlines",
    "W8,Cargojet Airways Ltd.",
    "W9,Eastwind Airlines Inc.",
    "WA,Worldwide Airlines Services",
    "WA (1),Western Air Lines Inc.",
    "WAC,Air East Airlines",
    "WAI,Wise Airlines Inc.",
    "WAP,Arrow Panama",
    "WAQ,Air Charter Systems (Acs)",
    "WAR,Waring Air",
    "WAW,Westward Airways Inc.",
    "WC,Wien Air Alaska Inc.",
    "WCA,Walker's International",
    "WCI,Mountainwest Airlines",
    "WD,Halisa Air",
    "WDQ,Wardair Canada Inc.",
    "WE,Centurion Cargo Inc.",
    "WEB,Southeast Alaska Airlines",
    "WEQ,Aero-Services-Corporate",
    "WES,Western Star Airlines Inc.",
    "WET,Western Charter Inc.",
    "WFS,Wheeler Airlines",
    "WFT,Westflight Aviation",
    "WG,Sunwing Airlines Inc.",
    "WGL,Wrangell Air Service",
    "WGQ,Gateway Aviation Limited",
    "WI,Sky Lease Cargo",
    "WIA,Wisc Air",
    "WIL,Will's Air",
    "WIR,Williams Air Inc.",
    "WL,Caribbean Sun Airlines, Inc. d/b/a World Atlantic Airlines",
    "WL (1),World Air Network",
    "WLH,Welch Aviation",
    "WLQ,Worldwide Airlines Inc.",
    "WM,Windward Islands Airways",
    "WN,Southwest Airlines Co.",
    "WNG,Wings Airways",
    "WNQ,Winair Inc.",
    "WO,World Airways Inc.",
    "WP,Island Air Hawaii",
    "WR,WestJet Encore Ltd.",
    "WRA,Wra Inc.",
    "WRD,Ward Air",
    "WRT,Wright Airlines Inc.",
    "WS,Westjet",
    "WS (1),Suncoast Airlines Inc.",
    "WSK,AAR Western Skyways",
    "WST,Friday Harbor Seaplanes",
    "WSU,Mission Air",
    "WT,Nigeria Airways Ltd.",
    "WTA,Westates Airlines",
    "WV,Air Greco, Inc., d/b/a Wings Air",
    "WV (1),Air South Inc. (1)",
    "WW,WOW Air ehf",
    "WWI,Wings West Airlines",
    "WWQ,Worldways Canada Ltd.",
    "X4,Air Excursions LLC",
    "X9,Omni Air International LLC",
    "XAA,Aero Union Corp",
    "XAB,Air Polynesia d/b/a DHL Cargo",
    "XAC,Air Express International Airlines",
    "XAE,Amerford Airways Inc.",
    "XAF,Astro-Wing Airlines Inc.",
    "XAG,Ball Brothers Inc.",
    "XAH,Hawaii Pacific Air",
    "XAI,Burlington Northern Inc.",
    "XAJ,Central American International",
    "XAK,Century Airlines Inc.",
    "XAM,Cohlmia Aviation",
    "XAO,Dallas Flight Center",
    "XAQ,Cam Air International Inc.",
    "XAR,Jet Way Inc.",
    "XAS,Klondike Air Inc.",
    "XAT,Michigan Peninsula Airways",
    "XAU,Flight Line Inc.",
    "XAY,Pat Air",
    "XAZ,Purolator Sky Courier Inc.",
    "XBC,Jet East Inc.",
    "XBF,WTC Airlines",
    "XBH,Profit Airlines Inc.",
    "XBI,CF Air Freight",
    "XBJ,International Air Service",
    "XBM,Phh Air Inc.",
    "XBP,Circle Airfreight Corp.",
    "XBQ,Aeronaves De Puerto Rico",
    "XBR,Allstates Air Cargo Inc.",
    "XBT,Pacific Alaska Airlines Inc.",
    "XBV,Nal Inc.",
    "XBX,Icb International Airlines",
    "XBZ,Air Natl Aircraft Sal & Ser",
    "XC,Air Caribbean (1)",
    "XCA,Hawkins And Powers Aviation",
    "XDA,Pacific Air Express Inc.",
    "XDC,Hemisphere Air Charter",
    "XDD,International Parcel Express Inc.",
    "XDE,Air Train Inc.",
    "XDF,Salair Inc.",
    "XDH,Kitty Hawk Air Cargo",
    "XE,ExpressJet Airlines Inc. (1)",
    "XF,Vladivostok Air",
    "XJ,Mesaba Airlines",
    "XK,Aerotal",
    "XL,Lan Ecuador",
    "XP,XTRA Airways",
    "XPA,Express Air d/b/a Sunrise",
    "XPQ,International Cargo Xpress",
    "XV,Presidential Airways Inc.",
    "XY,Munz Northern Airlines Inc.",
    "XYZ,Airpac Inc.",
    "Y2,Globespan Airways Limited d/b/a Flyglobespan",
    "Y4,Concesionaria Vuela Compania De Aviacion SA de CV (Volaris)",
    "Y7,Flyjet Limited d/b/a Silverjet",
    "Y8,Yangtze River Express Airlines Company",
    "YB,Trans Continental Airlines",
    "YBQ,Yellow Bird Air",
    "YM,Royal West Airlines",
    "YR,Grand Canyon Airlines, Inc. d/b/a Grand Canyon Airlines d/b/a Scenic Airlines",
    "YV,Mesa Airlines Inc.",
    "YX,Republic Airways",
    "YX (1),Midwest Airline, Inc.",
    "Z3,Bidzy Ta Hot Aana, Inc. d/b/a Tanana Air Service",
    "Z3 (1),Promech",
    "Z3Q,PM Air, LLC",
    "ZA,Accessair Holdings",
    "ZAQ,Zas Airline Of Egypt",
    "ZB,Monarch Airlines",
    "ZE,Eastar Jet Co Ltd.",
    "ZE (1),Lineas Aereas Azteca",
    "ZFZ,Mcphillips Flying Service",
    "ZIA,Air New Mexico",
    "ZK,Great Lakes Airlines",
    "ZKQ,Zantop International",
    "ZL,Affretair",
    "ZMZ,Winnipesaukee Aviation Inc.",
    "ZN,Key Airlines Inc.",
    "ZO,Florida Express Inc.",
    "ZP,Air St. Thomas",
    "ZS,Hispaniola Airways",
    "ZUQ,Zuliana De Aviacion",
    "ZV,Air Midwest Inc.",
    "ZW,Air Wisconsin Airlines Corp",
    "ZX,Air Georgian",
    "ZX (1),Airbc Ltd.",
    "ZY,Atlantic Gulf Airlines",
    "ZYZ,Skyway Aviation Inc."]

In [0]:
from pyspark.sql.functions import *

def init_data():
    print("Initializing data, the first run can take a few min to load and optimize all data...")
    if not spark.catalog.tableExists("flights"):
        print("Loading database flights...")
        initial_df = (spark.read.option("header", "true").option("inferSchema", "true").csv("dbfs:/databricks-datasets/airlines/part-00000"))
        df_schema = initial_df.schema
        #let's take only a subset to be faster
        df = spark.read.schema(df_schema).csv("dbfs:/databricks-datasets/airlines/part-019*").sample(0.1,42)
        newColumns = df.withColumn("DateString",concat(col("Year"),lit("-"),concat(col("Month"), lit("-"), col("DayofMonth")))).filter(col("Year").isNotNull())
        airlineDataTemp = newColumns.withColumn("Date", date_format(col("DateString"), "yyyy-MM-dd")).drop(col("DateString"))
        airlineDataTemp.write.partitionBy("Year", "UniqueCarrier").mode('overwrite').saveAsTable(f"{dbName}.flights")
        #spark.sql(f"""GRANT SELECT, MODIFY ON TABLE {dbName}.flights TO `account users`""")
        spark.sql(f"""ALTER TABLE {dbName}.flights OWNER TO `account users`""")
        #spark.sql("OPTIMIZE airlinedata.flights")

    if not spark.catalog.tableExists("lookupcodes"):
        print("Loading database lookupcodes...")
        codes = get_codes()
        codes_tuple = [((c.split(",")[0], c.split(",")[1])) for c in codes]
        deptColumns = ["dept_name","dept_id"]
        df_codes = spark.createDataFrame(data=codes_tuple, schema = ["UniqueCode", "Description"])
        df_codes.write.mode('overwrite').saveAsTable(f"{dbName}.lookupcodes")
        #spark.sql(f"""GRANT SELECT, MODIFY ON TABLE {dbName}.lookupcodes TO `account users`""")
        #spark.sql(f"""ALTER TABLE {dbName}.flights OWNER TO `account users`""")
        spark.sql(f"OPTIMIZE {dbName}.lookupcodes")

    if not spark.catalog.tableExists("airports"):
        print("Loading database airports...")
        airportsna = spark.read.format("com.databricks.spark.csv").options(header='true', inferschema='true', delimiter='\t').load("/databricks-datasets/flights/airport-codes-na.txt")
        airportsna.write.mode('overwrite').saveAsTable(f"{dbName}.airports")
        #spark.sql(f"""GRANT SELECT, MODIFY ON TABLE {dbName}.airports TO `account users`""")
        #spark.sql(f"""ALTER TABLE {dbName}.airports OWNER TO `account users`""")
        spark.sql(f"OPTIMIZE {dbName}.airports")
    
    
init_data()